In [4]:
# Module Imports

#DB stuff
import mariadb
import sys

#Encryption Stuff
import base64
from Crypto.Cipher import AES
import http.client
from Crypto import Random

#Other
import json
import time

In [7]:
#Connect with DB
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="airq",
        password="airq",
        host="localhost",
        port=3306,
        database="airq_data"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [ ]:
########################
airqIP = '192.168.4.1'
airqpass = 'airqsetup'
#########################

def unpad(data):
  return data[:-ord(data[-1])]


def decodeMessage(msgb64):
  # Erster Schritt: base64 dekodieren
  msg = base64.b64decode(msgb64)

  # AES-Schlüssel der Länge 32 aus dem air-Q-Passwort erstellen
  key = airqpass.encode('utf-8')
  if len(key) < 32:
    for i in range(32-len(key)):
      key += b'0'
  elif len(key) > 32:
    key = key[:32]

  # Zweiter Schritt: AES256 dekodieren
  cipher = AES.new(key=key, mode=AES.MODE_CBC, IV=msg[:16])
  return unpad(cipher.decrypt(msg[16:]).decode('utf-8'))

#Gibt ein JSON-Objekt über die verfügbaren Daten zurück
def getAvailable():
    # Verbindung zum air-Q aufbauen
    connection = http.client.HTTPConnection(airqIP)

    # Daten anfordern
    connection.request("GET", "/dirbuff")
    contents = connection.getresponse()

    # Daten entschlüsseln und ausgeben
    msg = json.loads(contents.read())
    msg['content'] = json.loads(decodeMessage(msg['content']))
    print(json.dumps(msg['content']))

    # Verbindung trennen
    connection.close()

    # Serializing json
    json_object = json.dumps(msg['content'], indent=4)
    
    # Writing to sample.json
    #with open("measurement_data.json", "w") as outfile:
    #    outfile.write(json_object)
    return json_object

In [ ]:
#Request available data

In [8]:
#DB-Connection close
conn.close()